## 数据处理脚本

In [1]:
import os
import pandas as pd
import json
import numpy as np
import random
data_path = 'data/'

In [2]:
from data import preProcess
total_item = {}
for filename in os.listdir(data_path):
    if('.xls' not in filename):
        continue
    df = pd.read_excel(data_path + filename)
    df.columns = [s.strip() for s in df.columns.tolist()]
    for row in df.index.values:
        id = str(df.loc[row, '工单编号'])
        title = df.loc[row, '诉求标题']
        if(not pd.isnull(df.loc[row, '市民原始诉求'])):
            text = str(df.loc[row, '市民原始诉求'])
        else:
            text = ('' if pd.isnull(df.loc[row, '涉事主体']) else '涉事主体:' + str(df.loc[row, '涉事主体'])) + \
                   ('' if pd.isnull(df.loc[row, '主体地址']) else '主体地址:' + str(df.loc[row, '主体地址'])) + \
                   ('' if pd.isnull(df.loc[row, '事发地点']) else '事发地点:' + str(df.loc[row, '事发地点'])) + \
                   ('' if pd.isnull(df.loc[row, '标签组']) else '标签组:' + str(df.loc[row, '标签组'])) + \
                   ('' if pd.isnull(df.loc[row, '市民诉求']) else '市民诉求:' + str(df.loc[row, '市民诉求'])) + \
                   ('' if pd.isnull(df.loc[row, '补充信息']) else '补充信息:' + str(df.loc[row, '补充信息']))
        text = preProcess(text)
        tag_level_1 = df.loc[row, '办理部门一级']
        tag_level_2 = df.loc[row, '办理部门二级']
        tag_level_3 = df.loc[row, '办理部门三级']
        total_item[str(id)] = {
            'title': title,
            'text': text,
            'tag_level_1': tag_level_1,
            'tag_level_2': tag_level_2,
            'tag_level_3': tag_level_3
        }
ids = list(total_item.keys())
random.shuffle(ids)
print(ids[:10])
train_item = {}
for id in ids[:len(ids) * 4 // 5]:
    train_item[id] = total_item[id]
test_item = {}
for id in ids[len(ids) * 4 // 5:]:
    test_item[id] = total_item[id]

mini_item = {}
for id in ids[:200]:
    mini_item[id] = total_item[id]

del total_item

['0322111312025281001', '032210171222311780101', '0322120218220039501', '0822111616562776101', '0322052122265540801', '2922102719444996401', '2822103108551591301', '0322051414390750501', '0822083012171772701', '0322112921221249301']


In [15]:
print(len(train_item))
print(len(test_item))
print(test_item[list(test_item.keys())[0]])

153320
38330
{'title': '反映人员管控问题', 'text': '问题类型:（其他影响公众健康的问题）\n其他问题:逸彩社区居委人员防疫不作为\n事发地址:荔湾区,广东省广州市荔湾区芳兴路逸彩新世界\n具体情况描述:广州疫情未停，今天是石围塘全员核酸，逸彩新世界小区内设有核酸点，人数众多。验核酸队经过小区架空层的老年人活动区，里面在打麻将的老人大部分都不戴口罩，存在传播风险，不符合现时的防疫要求。为此，我向 逸彩社区党群服务中心 的工作人员反映情况。\r\n然而，一位女性工作人员跟我说，她们之前有去说过的，但他们不听。（就这解释？那就是说明知道有疫情风险，还是不去跟进，认为说过一次就是尽责了？）然后该工作人员继续跟我说:“我们说过了，要不你去说说他们吧”…哈？我作为居民向你社区居委反映了问题，你居然叫我自己去处理？ 这是什么工作态度？你在党群服务中心是负责搞笑的吗？如此不作为！而且该工作人员没有佩戴口罩！\n希望相关部门如何处理:上级部门严肃教育批评 逸彩社区党群服务中心 工作人员的工作态度，严格落实各项防疫措施，加强工作人员自身素质。\n', 'tag_level_1': '荔湾区政府', 'tag_level_2': '石围塘街道办事处', 'tag_level_3': '石围塘街公共服务办'}


In [18]:
with open(data_path + 'train_set.json', 'w') as f:
    s = json.dumps(train_item)
    f.write(s)
with open(data_path + 'test_set.json', 'w') as f:
    s = json.dumps(test_item)
    f.write(s)

In [3]:
with open(data_path + 'mini_set.json', 'w') as f:
    s = json.dumps(mini_item)
    f.write(s)